# Pandas:

## the primary Python tabular data analysis/manipulation toolkit



### What is Pandas?

*    Python package
*    Functional toolkit
    *    Organizes data
    *    Manipulates data
    *    Analyzes data


In [ ]:
import pandas as pd
import numpy as np

##Speed!
* Built upon Numpy
* Cython core

###First data structure: Series

In [ ]:
s = pd.Series([1,2,3,3,4,5,4], index=['a','b','c','d','e','f','g'])
print(s)

###Slicing

In [ ]:
s[:3]

In [ ]:
(s + s) / 4

In [ ]:
s.get('f')

In [ ]:
s.get('y', np.nan)

###Second data structure: DataFrame
* Similar to R's data frame
* Does everything Series can do

In [ ]:
df = pd.read_csv('brothers.csv', index_col=0)

In [ ]:
print(df)

##Metadata

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame(
        [df['length_%s' % x] * df['breadth_%s' % x] for x in [1,2]],
            index=['brother 1', 'brother 2']
        )
df

In [ ]:
df = df.T
df.head()

In [ ]:
df = df / 1000
df.head()

##Boolean indexing

In [ ]:
df[df['brother 1'] < df['brother 2']]

In [ ]:
df.loc[:,'brother 2 greater?'] = df['brother 1'] < df['brother 2']
df.head()

##Sorting

In [ ]:
df.sort(columns='brother 1').head()

In [ ]:
df.sort_index(ascending=False).head()

##Iteration

In [ ]:
for index, data_series in df.iterrows():
    if index in [1,2]:
        print(index)
        print(data_series)

##SQL-like groupby operations

In [ ]:
df1 = pd.read_csv('mtcars.csv', index_col=0)
df1

In [ ]:
grouped = df1.groupby('cyl')

In [ ]:
for group_name, group in grouped:
    print(group_name)
    print(group)

##Plotting with matplotlib

In [ ]:
%matplotlib inline

In [ ]:
df1.plot(kind='scatter', x='cyl', y='hp')

##Function application

In [ ]:
df2 = pd.read_csv('data.csv')
df2

In [ ]:
df2.columns

In [ ]:
df3 = pd.read_csv('amino_acids.txt', header=None)
print(df3.head())

In [ ]:
import re
df3 = df3.loc[:,0].apply(lambda x: pd.Series(list(filter(None, re.compile('  +').split(x)))))
df3.head()

In [ ]:
df3.tail()

In [ ]:
df3 = df3.iloc[:-1,:]

In [ ]:
df3.columns = ['Abbreviation', 'First letter', 'Amino acid']

In [ ]:
df3.set_index('Amino acid', inplace=True)

In [ ]:
df3.head()

In [ ]:
print(df3.head())

In [ ]:
def change_names(name):
    if '_P' in name:
        name = df3.loc[name[:-2].replace('_', ' '), 'Abbreviation']
    return name

df2.columns = [change_names(name) for name in df2.columns]

In [ ]:
df2.set_index('ACCESSION', inplace=True)
df2.head()

##DataFrame columns are Series

In [ ]:
hcy_s = df2.loc[:,'Hcy']
hcy_s.head()

In [ ]:
hcy_s.unique()

In [ ]:
df2 = df2.drop('Hcy', axis=1)
df2.head()

In [ ]:
df2.loc[:,'Ala'].plot(kind='hist')

In [ ]:
df2.describe()

In [ ]:
age_s = df2.loc[:,'AGE_Years']
age_s.max()

In [ ]:
len(list(df2.loc[:,'AGE_Years'].unique()))

In [ ]:
df2.loc[:,'AGE_Years'] = age_s.round()
grouped = df2.groupby('AGE_Years')
grouped.count()

In [ ]:
def standard_score(df):
    return (df.mean() - df2.mean()) / df2.std()

df4 = grouped.get_group(4).pipe(standard_score) - \
    grouped.get_group(0).pipe(standard_score)
df4.drop('AGE_Years').plot(kind='bar')

In [ ]:
df2.drop(['AGE_Years', 'SEX', 'AA_Interp'], axis=1, inplace=True)
df2.apply(lambda x: x.sum())

In [ ]:
df2.apply(lambda x: x.sum(), axis=1)

In [ ]:
df2.applymap(lambda x: np.exp(x)).head()

##Concatenation

In [ ]:
data = pd.read_csv('HairEyeColor.csv', index_col=0)
data.head()

In [ ]:
df5 = data.loc[:,['Eye', 'Sex', 'Freq']][2:6]
df6 = data.loc[:,['Hair', 'Eye']][4:8]
df5

In [ ]:
df6

In [ ]:
df5 = pd.concat([df5,df6])
df5

##Handling missing data

In [ ]:
df5['Freq'].sum()

In [ ]:
df5.dropna(how='all', subset=['Freq', 'Sex'])

##Database-style merging
* Built for speed

In [ ]:
database = pd.read_csv('database.csv')
df7 = database.loc[:,['first_name', 'last_name', 'city', 'state']][2:6].reset_index(drop=True)
df8 = database.loc[:,['first_name', 'last_name', 'zip']][4:8].reset_index(drop=True)
df7

In [ ]:
df8

In [ ]:
df7.merge(df8, how='outer', on=['first_name', 'last_name'])

In [ ]:
df7.merge(df8, how='left', on=['first_name', 'last_name'])

In [ ]:
df7.merge(df8, how='right', on=['first_name', 'last_name'])

In [ ]:
df7.merge(df8, how='inner', on=['first_name', 'last_name'])

#Pandas vs. R

###R's c

In [ ]:
s_df = pd.DataFrame(np.random.randn(4, 3), columns=list('abc'))
s_df[['a', 'c']]

###Aggregation

In [ ]:
s_df = pd.DataFrame({
    'v1': [1,3,5,7,8,3,5,np.nan,4,5,7,9],
    'v2': [11,33,55,77,88,33,55,np.nan,44,55,77,99],
    'by1': ["red", "blue", 1, 2, np.nan, "big", 1, 2, "red", 1, np.nan, 12],
    'by2': ["wet", "dry", 99, 95, np.nan, "damp", 95, 99, "red", 99, np.nan,
           np.nan]
    })
s_df

In [ ]:
s_df.groupby(['by1','by2']).mean()

###R's %in%

In [ ]:
s = pd.Series(np.arange(5),dtype=np.float32)
s.isin([2, 4])

###R's match

In [ ]:
pd.Series(pd.match(s, [2,4], np.nan))

###R's tapply

In [ ]:
import random, string
baseball = pd.DataFrame({
    'team': ["team %d" % (x+1) for x in range(5)]*5,
    'player': random.sample(list(string.ascii_lowercase),25),
    'batting avg': np.random.uniform(.200, .400, 25)
    })
baseball.head()

In [ ]:
baseball.pivot_table(values='batting avg', columns='team', aggfunc=np.max)

###R's subset

In [ ]:
s_df = pd.DataFrame({'a': np.random.randn(10), 'b': np.random.randn(10)})
#s_df.query('a <= b')
s_df[s_df.a <= s_df.b]

###R's with

In [ ]:
#s_df.eval('a + b')
s_df.a + s_df.b

##rpy2: R data frame support

In [ ]:
from rpy2.robjects import r, pandas2ri
df_iris = pandas2ri.ri2py(r['iris'])
df_iris.head()

##Final comparison

* ###Pandas + Python
    * General-purpose programming language
    * Strong object-oriented capabilities
    * SciPy - library of scientific computing routines
    * scikit-learn - machine learning toolkit
* ###R
    * Very large library of statistical functions
    * Large statistics-oriented support base
    * Older thus wiser